# Real-World MDM: Mastering Hospital Data

In [ ]:
import pandas as pd

# Step 1 - Acquire the data and load it

Raw Datasets:

[**CMS** (Centers for Medicare & Medicaid Services)](https://www.cms.gov/Research-Statistics-Data-and-Systems/Downloadable-Public-Use-Files/Provider-of-Services/index) (Select the "Current California Healthcare Facility Listing" file)


[**OSHPD** (Office of Statewide Health Planning and Development)](https://data.chhs.ca.gov/dataset/licensed-healthcare-facility-listing) (Select the "POS Other CSV" file)


*Note: Apologies if my Python is clunky. I'm relatively new to it. If you can improve it, please submit a pull request or talk to me afterward. I'd love to learn!*

In [ ]:
oshpd = pd.read_csv("data/oshpd.csv")
cms = pd.read_csv("data/cms.csv")  #Note - I modified this file from raw. It was limited to only rows where STATE_CD = "CA"

In [ ]:
oshpd.head()

In [ ]:
cms.head()

# Step 2 - Analyze and Map the Data

*Woof, look at those column names. And 473 of them.*

***Tip #1:*** Know your data. Always ask for a data dictionary. (cms_data_dictionary.txt)

***Tip #1:*** Don't get distracted by unnecessary data.

In [ ]:
# After studying the data dictionary, we take just the columns that we need.

oshpd_simple = oshpd[[
    "OSHPD_ID",
    "FACILITY_NAME",
    "DBA_ADDRESS1",
    "DBA_CITY",
    "DBA_ZIP_CODE"]]

cms_simple = cms[[
    "PRVDR_NUM",
    "FAC_NAME",
    "ST_ADR",
    "CITY_NAME",
    "STATE_CD",
    "ZIP_CD"]]

In [ ]:
# Map OSHPD into a standard format

# Convert to a dictionaries for easier transforms
oshpd_dict = oshpd_simple.to_dict(orient="records")
cms_dict = cms_simple.to_dict(orient="records")

mapped = []

# Map OSHPD to a new list of dictionaries
for row in oshpd_dict:
    newrow = {
        "pkey":           "oshpd." + str(row["OSHPD_ID"]),
        "source":         "oshpd",
        "source_id":      row["OSHPD_ID"],
        "oshpd_id":       row["OSHPD_ID"],
        "name":           row["FACILITY_NAME"],
        "address_line_1": row["DBA_ADDRESS1"],
        "city":           row["DBA_CITY"],
        "region":         "CA",
        "post_code":      row["DBA_ZIP_CODE"]
    }
    mapped.append(newrow)
    
# Append CMS to that list of dictionaries
for row in cms_dict:
    newrow = {
        "pkey":           "cms." + str(row["PRVDR_NUM"]),
        "source":         "cms",
        "source_id":      row["PRVDR_NUM"],
        "cms_id":         row["PRVDR_NUM"],
        "name":           row["FAC_NAME"],
        "address_line_1": row["ST_ADR"],
        "city":           row["CITY_NAME"],
        "region":         row["STATE_CD"],
        "post_code":      row["ZIP_CD"]
    }
    mapped.append(newrow)

df = pd.DataFrame(mapped).set_index("pkey")[[
        "source",
        "source_id",
        "name",
        "oshpd_id",
        "cms_id",
        "address_line_1",
        "city",
        "region",
        "post_code"]]

In [ ]:
df.sample(10)

In [ ]:
# Do we see any dupes?

df.sort_values(by="name").head(20)

**Explore the Data**

_Do you see any duplicates?_

_How did you determine if two records were the same?_

# Step 3 - Cleanse

In [ ]:
# Eliminate periods in the name and address
df["name"] = df["name"].str.replace(".", "")
df["address_line_1"] = df["address_line_1"].str.replace(".", "")

# Lots more you can do, but we'll leave it at that

df.sample(20)

# Step 4 - Match

Every row needs to be compared to every other row to see if it's a strong match.

We'll run a simple match rule that says if Name + City + State are equal, they are a match.

In [ ]:
# Perform a self join, then limit the results to only those where Name, City, State are equal

df_noindex = df.reset_index()

matches = df_noindex.merge(df_noindex, how="inner", on=["name", "city", "region"])[["pkey_x", "pkey_y"]]
matches.head()

### A Mathy Aside....
This type of matching is an _equality relation_

Equality means:

* **Reflexive:** (A = A)
* **Symmetric:** A = B ==> B = A
* **Transitive:** A = B && B = C ==> A = C

We can see self-matches and symmetric matches in the data above. But that's not very useful, so let's get rid of the self matches and half of the symmetric matches.

Transitive matches create interesting "chains" of matches. Gaine has methods for eliminating them, which we won't dive into here.

In [ ]:
# pkey_x != pkey_y gets rid of self matches (A = A)
# pkey_x <= pkey_y gets rid of half the symmetric matches (allows A = B but not B = A)
# pkey_x < pkey_y does both

matches = matches[matches["pkey_x"] < matches["pkey_y"]]
matches.sample(10)

In [ ]:
# Let's display this in a stacked match report

# Queued records
queued = df.loc[matches["pkey_x"]].drop_duplicates().reset_index()
queued["record_src"] = "Queued"
queued["queued_pkey"] = queued["pkey"]

# Candidate Records
candidates = df.reset_index().merge(matches, how="inner", left_on="pkey", right_on="pkey_y")
candidates["record_src"] = "Candidate"
candidates["queued_pkey"] = candidates["pkey_x"]

# Union them
report = pd.concat([queued, candidates], sort=False)[["record_src", "queued_pkey", "pkey", "name", "address_line_1", "city", "region", "post_code", "oshpd_id", "cms_id"]] \
    .sort_values(by=["queued_pkey", "record_src"], ascending=[True, False])

report.head(20)

In [ ]:
report.to_excel("report.xlsx")